In [1]:
%matplotlib inline
import re
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, DateTime
from sqlalchemy.orm import sessionmaker
import sqlalchemy
import pickle

In [2]:
from konlpy.tag import *
hannanum = Hannanum()
kkma = Kkma()
twitter = Twitter()

In [3]:
codis = pd.read_csv('data/codis.csv')
items = pd.read_csv('data/items.csv')

In [ ]:
with open('pw.pw', 'rb') as f:
    pw = pickle.load(f) 

In [ ]:
ip = '13.125.171.94'
db = 'codi'

In [345]:
engine = sqlalchemy.create_engine("mysql+mysqldb://root:"+pw+"@"+ip+"/"+db)

# 코디 저장

In [371]:
codis = codis.sort_values('rank').reset_index(drop=True)

In [372]:
codis.to_sql(name='codis', con=engine, if_exists='replace', index_label='id')

# 태그 분석

In [348]:
corpus = []
#분리하기
for idx in range(len(codis)):
    
    st = codis['tag'][idx]

    try:
        res = re.split('[#\n ]+',st)
    except Exception as e:
        res = ['']
        print(st)
        
    if '' in res:
        res.remove('')
    res = list(set(res))
    
    corpus.append(' '.join(res))

nan
nan


In [349]:
from sklearn.feature_extraction.text import CountVectorizer
 
vect = CountVectorizer()
vect.fit(corpus)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [350]:
vc = vect.transform(corpus).toarray()
vc.shape

(1485, 1656)

In [383]:
tagmap = pd.DataFrame(vc, columns=vect.get_feature_names())

In [386]:
tagmap = pd.concat([codis['id_codi'], tagmap], axis=1)

In [388]:
tagmap.to_csv('map.csv', index=False)

In [351]:
rows = []
for idx in range(1485):
    tag1 = vc[idx]
    id_codi = codis.iloc[idx]['id_codi']
    idxs = np.argwhere(tag1==1).ravel().tolist()
    for cidx in idxs:
        rows.append([id_codi, cidx])

# 코디, 태그 매핑 정보 저장

In [352]:
mapping_codi_tag = pd.DataFrame(rows, columns=['id_codi','id_tag'])
mapping_codi_tag.to_sql(name='mapping_codi_tag',  con=engine, if_exists='replace', index_label='id')

# 태그 저장

In [353]:
tagdf = pd.DataFrame(vect.get_feature_names(), columns=['tag'])
tagdf.to_sql(name="tags", con=engine, if_exists='replace', index_label='id')

In [354]:
len(items['id_item'].value_counts())

2306

In [355]:
len(items)

6489

# 아이템 저장

In [358]:
def get_price(x):
    arr = re.split('[→()]', x)
    
    res = '' 
    if len(arr) == 1:
        res = x
    
    else:
         
        res = re.split('[→()]', x)[1]
      
    return res

In [359]:
items['price'] = items['price'].apply(get_price)
items['price'] = items['price'].astype(int)

In [360]:
mapping_codi_item = items.pivot_table(index=['id_codi', 'id_item']).reset_index().drop(columns=['price'])

In [361]:
mapping_codi_item.to_sql(name="mapping_codi_item", con=engine, if_exists='replace', index_label='id')

In [362]:
select * 
from mapping_codi 
join codis on mapping_codi.id_codi = codis.id_codi
join tags on mapping_codi.id_tag = tags.id
where codis.id_codi = '396015'


# select * 
# from `mapping_codi_item`
# join codis on `mapping_codi_item`.`id_codi` = `codis`.`id_codi`
# join items on `mapping_codi_item`.`id_item` = `items`.`id_item`
# where codis.id_codi= 395456

In [363]:
items = items.drop_duplicates(subset=['id_item']).reset_index(drop=True)

In [364]:
items.to_sql(name='items', con=engine, if_exists='replace', index_label='id')